In [6]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [ ]:
# setting up dataframe with columns required
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'link', 'rating'], index=None)

In [7]:
http = httplib2.Http()

In [ ]:
# the page links all follow pattern and can loop through nums to string just zero padded

# note from 2235 onwards there are a couple that do not have the standard codes in directory pages, will keep the standard code for now
# also some marked as missing from site or along those lines, probably added placeholder text, may want to add flag for easy filtering out

for n in range(1,7000):
  # timeout to limit load on the site (based on api requests per minute limit)
  if n % 240 == 0:
    print(n)
    time.sleep(60)

  # getting the number as a string in the way need to get url
  num_as_str = '{}{}'.format('0' * (3 - len(str(n))), str(n)) if n < 1000 else str(n)

  # requesting page for current/target scp, parsing with beautifulsoup
  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')

  # getting text for the scp within the main content div
  text = [i for i in soup.find(id = 'page-content').find_all('p')]

  # p tags include image captions which are excluded as below, may be other similar cases left in
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)

  # getting text from result set
  text = [i.text for i in text]

  # getting the rating - a few do not have this primarily the ones that are 'blocked' or similar
  rating = soup.find(class_ = 'number prw54353')
  if rating == None:
    pass
  else:
    rating = rating.text

  # accounting for different directory pages (used to get title) based on number - getting correct url below
  if n < 1000:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  else:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}'.format(str(int(n//1000)+1)))

  # requesting directory page
  soup = BeautifulSoup(response, 'html.parser')

  # some logic to get the text title (not scp number code) 
  # using next or previous siblings and numbers where needed as some have no code on the directory page or multiple codes
  if soup.find('a', text='SCP-{}'.format(num_as_str)) == None:
    # checks here due to cases where no previous or next sibling for one of the elements involved
    if soup.find('a', text='SCP-{}'.format(str(n-1))) != None and soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling() != None:
      title = soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling().text.split(" - ")[-1]
    else:
      title = soup.find('a', text='SCP-{}'.format(str(n+1))).parent.find_previous_sibling().text
      # removing the code part of titles that do have a code here
      if ' - ' in title:
        title = title.split(" - ")[-1]
    # could potentially add both codes for those that have two
  # basic title finder for most cases
  else:
    title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # quotes to preserve commas and not cause csv issues - joining up p tags with newlines
  text = "\"{}\"".format(" \n ".join(i for i in text))
  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), text, url, rating]
  scp_main_df.loc[len(scp_main_df)] = info



In [ ]:
scp_main_df.head

In [ ]:
scp_main_df.to_csv('scp6999.csv', index=False) 

In [10]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
import time

http = httplib2.Http()

In [15]:
# adding articles that were age restricted

# probably going to run just the numbers identified rather than loop everything and check each one for having a barrier
# can always incorporate this back into the main scrape

missing_numbers = ['SCP-001', 'SCP-231', 'SCP-597', 'SCP-695', 'SCP-835', 'SCP-1004', 'SCP-2182', 'SCP-2254', 'SCP-2607', 'SCP-2610', 'SCP-2721', 'SCP-3075', 
  'SCP-3099', 'SCP-3101', 'SCP-3236', 'SCP-3265', 'SCP-3380', 'SCP-3512', 'SCP-3629', 'SCP-3684', 'SCP-3726', 'SCP-3918', 'SCP-3929',
  'SCP-4106', 'SCP-4174', 'SCP-4203', 'SCP-4231', 'SCP-4269', 'SCP-4960', 'SCP-4969', 'SCP-5069', 'SCP-5169', 'SCP-5208', 'SCP-5328',
  'SCP-5342', 'SCP-5385', 'SCP-5444', 'SCP-5457', 'SCP-5468', 'SCP-5480', 'SCP-5488', 'SCP-5489', 'SCP-5558', 'SCP-5674', 'SCP-5690',
  'SCP-5706', 'SCP-5950', 'SCP-6090', 'SCP-6112', 'SCP-6127', 'SCP-6132', 'SCP-6142', 'SCP-6164', 'SCP-6220', 'SCP-6282', 'SCP-6312',
  'SCP-6328', 'SCP-6410', 'SCP-6423', 'SCP-6429', 'SCP-6465', 'SCP-6743', 'SCP-6771', 'SCP-6779', 'SCP-6802', 'SCP-6858', 'SCP-6908',
  'SCP-6952', 'SCP-6959', 'SCP-6961', 'SCP-6969', 'SCP-6972', 'SCP-6978', 'SCP-6985']

missing_numbers = map(lambda x: x.lower(), missing_numbers)
print(missing_numbers)

ratings_dict = {}
text_dict = {}

# just missing the text and rating, so can add these
for i in missing_numbers:
  url = 'https://scp-wiki.wikidot.com/adult:{}/noredirect/true'.format(i)

  status, response = http.request(url)
  soup = BeautifulSoup(response)

  rating_tag = soup.find(class_ = 'number prw54353')

  if rating_tag != None:
    rating = rating_tag.text  
    ratings_dict[i.upper()] = rating
  else:
    # looking at these that continue to be none show that there are some that are not blocked or age restricted and were likely just deleted
    ratings_dict[i.upper()] = None

  text = [i for i in soup.find(id = 'page-content').find_all('p')]

  for t in text:
    if hasattr(t.parent, 'class'):
      if t.parent.get("class") == 'scp-image-caption':
        text.remove(t)

  text = [i.text for i in text]

  text = "\"{}\"".format(" \n ".join(i for i in text))

  text_dict[i.upper()] = text



In [16]:
print(ratings_dict)
print(text_dict)

{'SCP-001': None, 'SCP-231': '+2165', 'SCP-597': '+131', 'SCP-695': '+88', 'SCP-835': '+872', 'SCP-1004': '+724', 'SCP-2182': '+241', 'SCP-2254': '+240', 'SCP-2607': '+126', 'SCP-2610': '+318', 'SCP-2721': None, 'SCP-3075': '+50', 'SCP-3099': '+209', 'SCP-3101': '+521', 'SCP-3236': '+238', 'SCP-3265': '+107', 'SCP-3380': '+186', 'SCP-3512': '+508', 'SCP-3629': '+133', 'SCP-3684': '+81', 'SCP-3726': None, 'SCP-3918': '+38', 'SCP-3929': '+405', 'SCP-4106': '+70', 'SCP-4174': '+46', 'SCP-4203': '+69', 'SCP-4231': '+428', 'SCP-4269': '+160', 'SCP-4960': '+590', 'SCP-4969': '+238', 'SCP-5069': '+208', 'SCP-5169': '+159', 'SCP-5208': '+62', 'SCP-5328': '+48', 'SCP-5342': None, 'SCP-5385': '+14', 'SCP-5444': '+82', 'SCP-5457': None, 'SCP-5468': None, 'SCP-5480': '+81', 'SCP-5488': '+86', 'SCP-5489': None, 'SCP-5558': '+196', 'SCP-5674': None, 'SCP-5690': '+244', 'SCP-5706': '+184', 'SCP-5950': '+78', 'SCP-6090': '+61', 'SCP-6112': None, 'SCP-6127': None, 'SCP-6132': None, 'SCP-6142': None, 'S